# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [3]:
%load_ext sql
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# pd.read_sql用のコネクタ
conn = psycopg2.connect(**pgconfig)
# pd.to_sql用のcreate engine
engine = create_engine(dsl)
# MagicコマンドでSQLを書くための設定
%sql $dsl

df_customer = pd.read_sql(sql='select * from customer', con=conn)
df_category = pd.read_sql(sql='select * from category', con=conn)
df_product = pd.read_sql(sql='select * from product', con=conn)
df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
df_store = pd.read_sql(sql='select * from store', con=conn)
df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [3]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1259971200,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1285113600,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1209859200,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1286668800,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [8]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [15]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].rename(columns={'sales_ymd' : 'sales_date'}).head(10)

,sales_date,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [20]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id == "CS018205000001"')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [30]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']] \
            .query('customer_id == "CS018205000001" and amount >= 1000')
# \はopt+¥

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [38]:
df_receipt[['sales_ymd','customer_id','product_cd','quantity','amount']] \
            .query('customer_id == "CS018205000001" & (amount >= 1000 | quantity >= 5)')

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [39]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']] \
            .query('customer_id=="CS018205000001" & 1000<=amount<=2000')

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [41]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']] \
            .query('customer_id == "CS018205000001" and product_cd != "P071401019"')

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [45]:
df_store.query('not(prefecture_cd == "13" | floor_area > 900)')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


In [157]:
df_store.query('prefecture_cd != "13" and floor_area <= 900')

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [125]:
df_store.query('store_cd.str.startswith("S14")', engine='python').head()
#.query()で文字列メソッドを使う場合、引数engine='python'を指定する必要がある

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0


---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [155]:
df_customer.query('customer_id.str.endswith("1")', engine='python').head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4


---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [161]:
df_store.query('address.str.contains("横浜市")',engine='python').head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0


---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [122]:
df_customer.query('status_cd.str.contains("^[A-F]", regex=True)',engine='python').head()

# ^[A-F]は正規表現。メタ文字^は行の先頭を指定、[A-F]はAからFのどれか。=>　「行の先頭に、AからF」をstr.contains()という感じ。
# https://techacademy.jp/magazine/15635#:~:text=Python%E3%81%AE%E6%AD%A3%E8%A6%8F%E8%A1%A8%E7%8F%BE%E3%81%A8,%E8%A1%8C%E3%81%86%E3%81%93%E3%81%A8%E3%81%8C%E3%81%A7%E3%81%8D%E3%81%BE%E3%81%99%E3%80%82

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F


---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [121]:
df_customer.query('status_cd.str.contains("[1-9]$", regex=True)', engine='python').head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1


---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [126]:
df_customer.query('status_cd.str.contains("^[A-F].*[1-9]$", regex=True)', engine='python').head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3


---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [127]:
df_store.query('tel_no.str.contains("[0-9]{3}-[0-9]{3}-[0-9]{4}", regex=True)', engine='python').head()

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [128]:
df_customer.sort_values('birth_day', ascending=True).head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0


---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [129]:
df_customer.sort_values('birth_day', ascending=False).head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1


---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [146]:
#模範回答
df_tmp = pd.concat([df_receipt[['customer_id','amount']] ,df_receipt['amount'].rank(method='min', ascending=False)], axis=1)
df_tmp.columns = ['customer_id','amount', 'ranking']
df_tmp.sort_values('ranking', ascending=True).head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


In [29]:
#自作
ranking =df_receipt['amount'].rank(method='min', ascending=False)
df_tmp = pd.concat([df_receipt[['customer_id','amount']],ranking],axis=1)
df_tmp.columns = ['customer_id', 'amount', 'ranking']
df_tmp.sort_values('ranking', ascending=True).head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
72747,ZZ000000000000,5480,4.0
10320,ZZ000000000000,5480,4.0
97294,CS021515000089,5440,7.0
28304,ZZ000000000000,5440,7.0
92246,CS009415000038,5280,9.0
68553,CS040415000200,5280,9.0


---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [33]:
df_tmp = pd.concat([df_receipt[['customer_id','amount']],df_receipt['amount'].rank(method='first', ascending=False)],axis=1)
df_tmp.columns = ['customer_id','amount','ranking']
df_tmp.sort_values('ranking', ascending=True).head(10)

,customer_id,amount,ranking
1202,CS011415000006,10925,1.0
62317,ZZ000000000000,6800,2.0
54095,CS028605000002,5780,3.0
4632,CS015515000034,5480,4.0
10320,ZZ000000000000,5480,5.0
72747,ZZ000000000000,5480,6.0
28304,ZZ000000000000,5440,7.0
97294,CS021515000089,5440,8.0
596,CS015515000083,5280,9.0
11275,CS017414000114,5280,10.0


---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [34]:
len(df_receipt)

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [36]:
df_receipt['customer_id'].nunique()
#または
#len(df_receipt['customer_id'].unique())   *nunique　か　uniqueの違い

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [45]:
df_receipt.groupby('store_cd').agg({'amount':sum, 'quantity':sum}).head()
#　回答編は、.reset_index()メソッドをケツにつけてindex振り直してた

,amount,quantity
store_cd,,
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403


---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [53]:
df_receipt.groupby('customer_id').sales_ymd.max().reset_index().head()

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224


---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [54]:
df_receipt.groupby('customer_id').sales_ymd.min().reset_index().head()

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20180503
2,CS001115000010,20171228
3,CS001205000004,20170914
4,CS001205000006,20180207


In [84]:
df_receipt.groupby('customer_id').agg({'sales_ymd':'min'}).head()

,sales_ymd
customer_id,
CS001113000004,20190308
CS001114000005,20180503
CS001115000010,20171228
CS001205000004,20170914
CS001205000006,20180207


---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [130]:
df_tmp = df_receipt.groupby('customer_id').agg({'sales_ymd':['max','min']}).reset_index()
df_tmp.columns = ['_'.join(pair) for pair in df_tmp.columns]
df_tmp.query('sales_ymd_max != sales_ymd_min').head()

,customer_id_,sales_ymd_max,sales_ymd_min
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
13,CS001214000009,20190902,20170306


---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [82]:
df_receipt.groupby('store_cd').agg({'amount':'mean'}).sort_values('amount',ascending=False).reset_index().head()

,store_cd,amount
0,S13052,402.867470
1,S13015,351.111960
2,S13003,350.915519
3,S14010,348.791262
4,S13001,348.470386


---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [85]:
df_receipt.groupby('store_cd').agg({'amount':'median'}).sort_values('amount', ascending=False).head()

,amount
store_cd,
S13052,190
S14010,188
S14050,185
S14040,180
S13003,180


---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [101]:
#模範回答  
df_receipt.groupby('store_cd').product_cd.apply(lambda x: x.mode()).reset_index().head()

#.apply()は列内の各要素に関数を当てる。この場合各要素xに.mode()を当てて、最頻値を返している。axis=1にすると行の各要素に当てる。

,store_cd,level_1,product_cd
0,S12007,0,P060303001
1,S12013,0,P060303001
2,S12014,0,P060303001
3,S12029,0,P060303001
4,S12030,0,P060303001


---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [119]:
df_receipt.groupby('store_cd').amount.var(ddof=0).reset_index().sort_values('amount',ascending=False).head()

,store_cd,amount
28,S13052,440088.701311
31,S14011,306314.558164
42,S14034,296920.081011
5,S13001,295431.993329
12,S13015,295294.361116


---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [118]:
df_receipt.groupby('store_cd').amount.std(ddof=0).reset_index().sort_values('amount',ascending=False).head()

,store_cd,amount
28,S13052,663.391816
31,S14011,553.456916
42,S14034,544.903736
5,S13001,543.536561
12,S13015,543.409938


---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [137]:
df_receipt['amount'].quantile([0,0.25,0.5,0.75,1.0])
# 模範回答
# df_receipt.amount.quantile(np.arange(5)/4)　*[0,1,2,3,4]のリストを４で割る

0.00       10.0
0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [145]:
df_receipt.groupby('store_cd').amount.mean().reset_index().query('amount >= 330').head()

,store_cd,amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960


---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [169]:
df_receipt.query('not(customer_id.str.startswith("Z"))',engine='python')\
            .groupby('customer_id').amount.sum().mean()

2547.742234529256

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [188]:
amount_mean = df_receipt.query('not customer_id.str.startswith("Z")',engine='python').groupby('customer_id').amount.sum().mean()
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_amount_sum.query('amount >= @amount_mean').head()
#回答例
#df_amount_sum[df_amount_sum['amount'] >= amount_mean].head(10)

,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639


---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [194]:
df_receipt.merge(df_store[['store_cd','store_name']],on='store_cd',how='inner').head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
1,20181116,1258329600,S14006,112,2,ZZ000000000000,P080401001,1,48,葛が谷店
2,20170118,1200614400,S14006,1162,1,CS006815000006,P050406035,1,220,葛が谷店
3,20190524,1274659200,S14006,1192,1,CS006514000034,P060104003,1,80,葛が谷店
4,20190419,1271635200,S14006,112,2,ZZ000000000000,P060501002,1,148,葛が谷店


---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [211]:
df_product.merge(df_category[['category_major_cd', 'category_medium_cd',
                              'category_small_cd','category_small_name']]
                 ,on=['category_major_cd', 'category_medium_cd',
                              'category_small_cd']
                 ,how='inner').head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,弁当類
1,P040101002,04,0401,040101,218.0,164.0,弁当類
2,P040101003,04,0401,040101,230.0,173.0,弁当類
3,P040101004,04,0401,040101,248.0,186.0,弁当類
4,P040101005,04,0401,040101,268.0,201.0,弁当類


---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [17]:
df_amount_sum= df_receipt.groupby('customer_id').amount.sum().reset_index()
df_tmp = df_customer.query('gender_cd == "1" & not customer_id.str.startswith("Z")',engine='python')
pd.merge(df_tmp['customer_id'], df_amount_sum, how='left', on='customer_id').fillna(0).head()

,customer_id,amount
0,CS021313000114,0.0
1,CS031415000172,5088.0
2,CS028811000001,0.0
3,CS001215000145,875.0
4,CS015414000103,3122.0


---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [50]:
df_tmp = df_receipt[~df_receipt['customer_id'].str.startswith("Z")]
df_sum = df_tmp.groupby('customer_id').amount.sum().reset_index().sort_values('amount',ascending=False).head(20)

df_frq = df_tmp[~df_tmp.duplicated(keep='first', subset=['customer_id','sales_ymd'])]\
            .groupby('customer_id').sales_ymd.count().reset_index().sort_values('sales_ymd',ascending=False).head(20)
pd.merge(df_sum, df_frq, on='customer_id', how='outer').head()

,customer_id,amount,sales_ymd
0,CS017415000097,23086.0,20.0
1,CS015415000185,20153.0,22.0
2,CS031414000051,19202.0,19.0
3,CS028415000007,19127.0,21.0
4,CS001605000009,18925.0,NaN


---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [59]:
df_store_tmp=df_store.copy()
df_product_tmp=df_product.copy()
df_store_tmp['dummy']=(0)
df_product_tmp['dummy']=(0)
len(pd.merge(df_store_tmp,df_product_tmp, how ='outer', on ='dummy'))

531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [66]:
df_tmp = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
df_tmp['amount_diff']=df_tmp['amount'].diff()
df_tmp.head(5)

,sales_ymd,amount,amount_diff
0,20170101,33723,NaN
1,20170102,24165,-9558.0
2,20170103,27503,3338.0
3,20170104,36165,8662.0
4,20170105,37830,1665.0


---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [87]:
df_tmp = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
df_tmp = pd.concat([df_tmp, df_tmp.shift(), df_tmp.shift(2), df_tmp.shift(3)], axis=1)
df_tmp.columns= ['sales_ymd_N', 'amount_N'
                 , 'sales_ymd_N-1', 'amount_N-1'
                 , 'sales_ymd_N-2', 'amount_N-2'
                 , 'sales_ymd_N-3', 'amount_N-3']
df_tmp.dropna().head(10)

,sales_ymd_N,amount_N,sales_ymd_N-1,amount_N-1,sales_ymd_N-2,amount_N-2,sales_ymd_N-3,amount_N-3
3,20170104,36165,20170103.0,27503.0,20170102.0,24165.0,20170101.0,33723.0
4,20170105,37830,20170104.0,36165.0,20170103.0,27503.0,20170102.0,24165.0
5,20170106,32387,20170105.0,37830.0,20170104.0,36165.0,20170103.0,27503.0
6,20170107,23415,20170106.0,32387.0,20170105.0,37830.0,20170104.0,36165.0
7,20170108,24737,20170107.0,23415.0,20170106.0,32387.0,20170105.0,37830.0
8,20170109,26718,20170108.0,24737.0,20170107.0,23415.0,20170106.0,32387.0
9,20170110,20143,20170109.0,26718.0,20170108.0,24737.0,20170107.0,23415.0
10,20170111,24287,20170110.0,20143.0,20170109.0,26718.0,20170108.0,24737.0
11,20170112,23526,20170111.0,24287.0,20170110.0,20143.0,20170109.0,26718.0
12,20170113,28004,20170112.0,23526.0,20170111.0,24287.0,20170110.0,20143.0


---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [161]:
df_tmp = pd.merge(df_receipt, df_customer, how ='inner', on="customer_id")
df_tmp['era'] = df_tmp['age'].apply(lambda x: math.floor(x / 10) * 10)
df_sales_summary = pd.pivot_table(df_tmp, index='era', columns='gender_cd', values='amount', aggfunc='sum').reset_index()
df_sales_summary.columns = ['era', 'male', 'female', 'unknown']
df_sales_summary

,male,female,unknown,total
era,,,,
10,1591.0,149836.0,4317.0,155744
20,72940.0,1363724.0,44328.0,1480992
30,177322.0,693047.0,50441.0,920810
40,19355.0,9320791.0,483512.0,9823658
50,54320.0,6685192.0,342923.0,7082435
60,272469.0,987741.0,71418.0,1331628
70,13435.0,29764.0,2427.0,45626
80,46360.0,262923.0,5111.0,314394
90,NaN,6260.0,NaN,6260


---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [ ]:
df_sales_summary2 = df_sales_summary.set_index('era').stack().reset_index() \
    .replace({'male':'00','female':'01','unknown':'99'})
#set_index()は特定の列をindexに。stack()は列を行方向にピボット。
df_sales_summary2.columns = ['era','gender_cd','amount']
df_sales_summary2

---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
print(df_customer['birth_day'].dtypes)
print(type(df_customer['birth_day'][0]))
pd.concat([df_customer['customer_id'],pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y%m%d')],axis=1).head()

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_customer['customer_id'],pd.to_datetime(df_customer['application_date'])], axis=1).head()


---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_ymd'].astype(str))],axis=1).head()

---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch'],unit='s')],axis=1).head()

---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.year],axis=1).head()

---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.strftime('%m')],axis=1).head()

---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']], pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%d')],axis=1).head()

---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_tmp = df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_flg']= df_tmp['amount'].apply(lambda x: 0 if x <= 2000 else 1)
df_tmp.head()

---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [ ]:
df_customer['area_flg']=df_customer['postal_cd'].apply(lambda x: 1 if 100 <= int(x[0:3]) <= 209 else 0)
pd.merge(df_receipt, df_customer[['customer_id','area_flg']], on='customer_id', how='inner').groupby('area_flg').agg({'customer_id': 'nunique'})

---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [ ]:
df_customer['pref_cd']= df_customer['address'].str[0:3].map({'埼玉県': '11', '千葉県': '12', '東京都': '13', '神奈川': '14'})
df_customer[['customer_id','address','pref_cd']].head()

---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [ ]:
df_tmp=df_receipt[['customer_id','amount']].groupby('customer_id').amount.sum().reset_index()
q1 = df_tmp['amount'].quantile(0.25)
q2 = df_tmp['amount'].quantile(0.5)
q3 = df_tmp['amount'].quantile(0.75)

df_tmp['amount_flg']= df_tmp['amount'].apply(lambda x: 1 if x < q1 else 2 if q1<= x < q2 else 3 if q2<= x < q3 else 4)
df_tmp.head()

In [ ]:
df_tmp_2=df_receipt[['customer_id','amount']].groupby('customer_id').amount.sum().reset_index()
q4 = np.quantile(df_tmp_2['amount'], 0.25)
q5 = np.quantile(df_tmp_2['amount'], 0.5)
q6 = np.quantile(df_tmp_2['amount'], 0.5)

def pct_group(x):
    if x < q4:
        return 1
    elif q4<= x < q5:
        return 2
    elif q5<= x < q6:
        return 3
    else:
        return 4

df_tmp_2['amount_flg']= df_tmp_2['amount'].apply(lambda x: pct_group(x))
df_tmp_2.head()

---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [ ]:
df_customer['era']=pd.cut(df_customer['age'], bins=[0,10,20,30,40,50,60,np.inf], right=False)
df_customer[['customer_id','birth_day','era']].head()

In [ ]:
df_customer['era']=df_customer['age'].apply(lambda x: min(math.floor(x/10)*10, 60))
df_customer[['customer_id','birth_day','era']].head()

In [ ]:
df_customer['era']=df_customer['age'].apply(lambda x: min(math.floor(x/10)*10, 60))
df_customer[['customer_id','birth_day','era']].head()

---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [ ]:
df_tmp = df_customer[['customer_id','birth_day','era']]
df_tmp['era_gender']= df_customer['era'].astype(str) + '代'+ df_customer['gender']
df_tmp['era_gendercd']= df_customer['era'].astype(str) + '_'+ df_customer['gender_cd'].astype(str)
df_tmp.head()

---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [ ]:
pd.get_dummies(df_customer[['customer_id','gender_cd']], columns=['gender_cd']).head()

---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp= df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_ss'] = preprocessing.scale(df_tmp['amount'])
df_tmp.head()

---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp=df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_mm']=preprocessing.minmax_scale(df_tmp['amount'])
df_tmp.describe()

---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp=df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_log10']=[math.log10(x) for x in df_tmp['amount']]
df_tmp.head()

---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_tmp= df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_tmp['amount_loge']= [math.log(x,math.e) for x in df_tmp['amount']]
df_tmp.head()

---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [ ]:
df_product['unit_profit']=df_product['unit_price']-df_product['unit_cost']
df_product.head()

---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [ ]:
(1 - df_product['unit_cost']/df_product['unit_price']).mean(skipna=True)

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()
df_tmp['new_price']= (df_tmp['unit_cost']/ 0.7).apply(lambda x : np.floor(x)) #math.floorは欠損値エラー起こる
df_tmp['profit_rate']= (1 - df_tmp['unit_cost']/df_tmp['new_price']).head()
df_tmp.head()

---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()
df_tmp['new_price']= (df_tmp['unit_cost']/ 0.7).round()
df_tmp['profit_rate']= (1 - df_tmp['unit_cost']/df_tmp['new_price']).head()
df_tmp.head()

---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
# math.ceilはNaNでエラーとなるが、numpy.ceilはエラーとならない
df_tmp=df_product.copy()
df_tmp['new_price']= df_tmp['unit_cost'].apply(lambda x : np.ceil(x/0.7)) #math.ceil()は欠損値エラー起きる
df_tmp['profit_rate']= (1 - df_tmp['unit_cost']/df_tmp['new_price']).head()
df_tmp.head()

---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [ ]:
# math.floorはNaNでエラーとなるが、numpy.floorはエラーとならない
df_tmp= df_product.copy()
df_tmp['price_taxinc']= df_tmp['unit_price'].apply(lambda x: np.floor(x*1.1))
df_tmp.head()

---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [ ]:
df_tmp= pd.merge(df_receipt,df_product)
df_tmp_a= df_tmp.groupby('customer_id').amount.sum().reset_index()
df_tmp_b= df_tmp[df_tmp['category_major_cd']==7].groupby('customer_id').amount.sum().reset_index()
df_tmp= pd.merge(df_tmp_a,df_tmp_b, on='customer_id', how='right')
df_tmp.assign(y_rate= df_tmp['amount_y']/df_tmp['amount_x']).head()

---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
df_tmp= pd.merge(df_receipt[['sales_ymd','customer_id']],df_customer[['customer_id','application_date']], on='customer_id',how='inner')

df_tmp['sales_ymd']= pd.to_datetime(df_tmp['sales_ymd'], format='%Y%m%d')
df_tmp['application_date']= pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

df_tmp['days']= df_tmp['sales_ymd'] - df_tmp['application_date']

df_tmp.head()

---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
df_tmp=pd.merge(df_receipt[['sales_ymd','customer_id']], df_customer[['customer_id','application_date']], on='customer_id', how='inner')

df_tmp['sales_ymd']= pd.to_datetime(df_tmp['sales_ymd'], format='%Y%m%d')
df_tmp['application_date']= pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

df_tmp['months'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x: relativedelta(x[0], x[1]).years * 12 + relativedelta(x[0], x[1]).months, axis=1)

df_tmp.head()

---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
df_tmp=pd.merge(df_receipt[['sales_ymd','customer_id']], df_customer[['customer_id','application_date']], on='customer_id', how='inner')

df_tmp['sales_ymd']= pd.to_datetime(df_tmp['sales_ymd'], format='%Y%m%d')
df_tmp['application_date']= pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

df_tmp['years'] = df_tmp[['sales_ymd', 'application_date']].apply(lambda x: relativedelta(x[0], x[1]).years, axis=1)

df_tmp.head()

---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
df_tmp=pd.merge(df_receipt[['sales_ymd','customer_id']], df_customer[['customer_id','application_date']], on='customer_id', how='inner')
df_tmp['sales_ymd']= pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date']= pd.to_datetime(df_tmp['application_date'], format='%Y%m%d')

df_tmp['elapsed_date']= (df_tmp['sales_ymd'] - df_tmp['application_date']).dt.total_seconds()

#解答例
#df_tmp['elapsed_date']= (df_tmp['sales_ymd'].astype('int')/10**9) - (df_tmp['application_date'].astype('int')/10**9)

df_tmp.head()

---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
df_tmp = df_receipt[['customer_id','sales_ymd']]
df_tmp['sales_ymd']= pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['monday']= df_tmp['sales_ymd'].apply(lambda x: x - relativedelta(days=x.weekday()))
df_tmp['from_monday']= df_tmp[['sales_ymd','monday']].apply(lambda x : relativedelta(x[0],x[1]).days, axis=1)
df_tmp

---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [ ]:
df_customer.sample(frac=0.01, random_state=0).head()

---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [ ]:
_, df_tmp = train_test_split(df_customer, test_size=0.1, stratify=df_customer['gender_cd'])
df_tmp.groupby('gender_cd').agg({'customer_id':'count'})
#df_tmp.groupby('gender_cd').customer_id.count().reset_index()

---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [ ]:
# 3σ：標準偏差σの3個分 ⇒ グループ化後の'amount.sum()'を標準化して比較
df_tmp= df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_tmp['amount_ss']= preprocessing.scale(df_tmp['amount'])
df_tmp.query('abs(amount_ss) >= 3').head()

---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [ ]:
df_tmp= df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').agg({'amount':'sum'})

q1= df_tmp['amount'].quantile(0.25)
q3= df_tmp['amount'].quantile(0.75)
iqr= q3 - q1

a= q1 - (iqr * 1.5)
b= q3 + (iqr * 1.5)

df_tmp.query('not @a <= amount <= @b').head()

---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
df_product.isnull().sum()

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [ ]:
df_product1 = df_product.copy()
print(len(df_product1))
df_product1.dropna(inplace=True)
print(len(df_product1))

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_product2= df_product.fillna({'unit_price':round(np.nanmean(df_product['unit_price'])),\
                                'unit_cost':round(np.nanmean(df_product['unit_cost']))})
df_product2.isnull().sum()

In [ ]:
df_product2 = df_product.copy()
up_mean= round(df_product2.unit_price.mean())
uc_mean= round(df_product2.unit_cost.mean())
df_product2= df_product2.fillna({'unit_price':up_mean,'unit_cost':uc_mean})
df_product2.isnull().sum()

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_product3= df_product.fillna({'unit_price': round(np.nanmedian(df_product['unit_price'])), 'unit_cost': round(np.nanmedian(df_product['unit_cost']))})
df_product3.isnull().sum()

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
df_tmp= df_product.groupby('category_small_cd').agg({'unit_price':'median','unit_cost':'median'}).reset_index()
df_tmp.columns= ['category_small_cd','price_median','cost_median']
df_product4= pd.merge(df_product,df_tmp, on='category_small_cd', how='inner')

df_product4['unit_price']= df_product4[['unit_price','price_median']].apply(lambda x: round(x[1]) if np.isnan(x[0]) else x[0], axis=1)
df_product4['unit_cost']= df_product4[['unit_cost','cost_median']].apply(lambda x: round(x[1]) if np.isnan(x[0]) else x[0], axis=1)

df_product4.isnull().sum()

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [ ]:
df_tmp= pd.merge(df_customer['customer_id'],df_receipt[['customer_id','sales_ymd','amount']],on='customer_id',how='left').fillna(0)
df_tmp1= df_tmp.groupby('customer_id').amount.sum().reset_index()
df_tmp2= df_tmp[df_tmp['sales_ymd'].astype('str').str.startswith("2019")].groupby('customer_id').amount.sum().reset_index()
df_tmp1.columns = ['customer_id','amount_all']
df_tmp2= df_tmp2.rename(columns= {'amount':'amount_2019'})
df_amount= pd.merge(df_tmp1,df_tmp2, on='customer_id', how='left').fillna(0)
df_amount['rate_2019']= (df_amount['amount_2019'] / df_amount['amount_all']).fillna(0)

print(df_amount.isnull().sum())
df_amount[df_amount['rate_2019'] > 0].head()

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [ ]:
#postal_cdに対して複数の緯度経度がある場合がある（1丁目と2丁目とか）
print([df_geocode['postal_cd'].nunique(),len(df_geocode['postal_cd'])]) #確認用

df_customer_1 = pd.merge(df_customer[['customer_id','postal_cd']],df_geocode[['postal_cd','longitude','latitude']], on='postal_cd', how='inner')

#postal_cdに対して複数の緯度経度がある場合、inner結合でもcustomer_idはその分複製される。
print([df_customer_1['customer_id'].nunique(),len(df_customer_1['customer_id'])]) #確認用

df_customer_1 = df_customer_1.groupby('customer_id').agg({'longitude':'mean','latitude':'mean'}).reset_index().rename(columns={'longitude':'longitude_mean','latitude':'latitude_mean'})

print(len(df_customer_1['customer_id'])) #確認用
df_customer_1= pd.merge(df_customer,df_customer_1, on='customer_id', how='inner')
df_customer_1.head(2)

---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [ ]:
#地点間の距離を求める関数を設定
def calc_distance(x1, y1, x2, y2):
    distance = 6371 * math.acos(math.sin(math.radians(y1)) * math.sin(math.radians(y2)) \
                                + math.cos(math.radians(y1)) * math.cos(math.radians(y2)) \
                                * math.cos(math.radians(x1) - math.radians(x2)))
    return distance

df_customer_2 = pd.merge(df_customer_1,df_store, left_on='application_store_cd',right_on='store_cd', how='inner')

df_customer_2['distance']= df_customer_2[['longitude_mean','latitude_mean','longitude','latitude']].apply(lambda x: calc_distance(x[0],x[1],x[2],x[3]), axis=1)

df_customer_2[['customer_id','address_x','address_y','distance']].head()

---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [ ]:
df_tmp = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_customer_u = pd.merge(df_customer,df_tmp,on='customer_id',how='left').sort_values(['amount','customer_id'],ascending=[False, True])
#複数列のsort_valuesは、リストの後ろ→前の順にソートされていく = 結果、amount最優先で、同一値はcustomer_id順
df_customer_u.drop_duplicates(subset=['customer_name','postal_cd'], keep='first', inplace=True)
print([len(df_customer),len(df_customer_u)])

---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [ ]:
df_customer_n = pd.merge(df_customer,df_customer_u[['customer_id','customer_name','postal_cd']], on=['customer_name','postal_cd'], how='inner').rename(columns={'customer_id_x':'customer_id','customer_id_y':'integration_id'})
print([df_customer_n['customer_id'].nunique(),df_customer_n['integration_id'].nunique()])

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

In [ ]:
del df_customer_1
del df_customer_n

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [ ]:
df_tmp = pd.merge(df_customer,df_receipt, on='customer_id', how='inner')
df_train, df_test = train_test_split(df_tmp, test_size = 0.2, random_state=1)
len(df_train)/len(df_tmp)

---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [ ]:
df_tmp= df_receipt.copy()
df_tmp['sales_ym']= df_tmp['sales_ymd'].astype('str').str[0:6]
df_tmp= df_tmp.groupby('sales_ym').amount.sum().reset_index()

def split_data(df, train_size, test_size, slide_window, start_point):
    train_start = start_point * slide_window
    test_start = train_start + train_size
    return df[train_start : test_start], df[test_start : test_start + test_size]

train_1, test_1 = split_data(df=df_tmp, train_size=12, test_size=6, slide_window=6, start_point=0)
train_2, test_2 = split_data(df_tmp, 12, 6, 6, 1)
train_3, test_3 = split_data(df_tmp, 12, 6, 6, 2)

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [ ]:
df_tmp= df_receipt.groupby('customer_id').amount.sum().reset_index()

df_tmp= pd.merge(df_customer, df_tmp, on='customer_id', how='left')

df_tmp['sales_flg']= df_tmp['amount'].apply(lambda x: 0 if np.isnan(x) else 1)

print('0の件数: ',len(df_tmp[df_tmp['sales_flg']==0]))
print('1の件数: ',len(df_tmp[df_tmp['sales_flg']==1]))

positive_count= len(df_tmp[df_tmp['sales_flg']==1])
rs= RandomUnderSampler(random_state=1)

df_sample, _ = rs.fit_sample(df_tmp, df_tmp.sales_flg)

print('0の件数: ',len(df_sample[df_sample['sales_flg']==0]))
print('1の件数: ',len(df_sample.query('sales_flg == 1')))

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [ ]:
df_gender=df_customer[['gender_cd','gender']].drop_duplicates().sort_values('gender_cd', ascending=True).reset_index(drop=True)

df_customer_new= df_customer.drop(columns='gender')

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [ ]:
df_tmp= pd.merge(df_product, df_category, how='inner')
df_tmp.head(2)

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
os.makedirs('./data')
df_tmp.to_csv('./data/df_tmp.csv', encoding='utf-8', header=True, index=False, sep=',')

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [ ]:
df_tmp.to_csv('data/df_tmp_2.csv', index=False, encoding='cp932')

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [ ]:
df_tmp.to_csv('data/df_tmp_3.csv', header=False)

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_tmp4= pd.read_csv('./data/df_tmp.csv')
df_tmp4.head()

---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [ ]:
df_tmp5= pd.read_csv('./data/df_tmp_3.csv',header = None)
df_tmp5.head()

---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_tmp.to_csv('data/df_tmp6.tsv', index=False, sep='\t')

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_tmp6=pd.read_table('data/df_tmp6.tsv')
df_tmp6.head()

# これで１００本終わりです。おつかれさまでした！